In [ ]:
import Bio.Restriction as Restriction
import pygsheets
import requests
import toml
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.geneious as geneious
import paulssonlab.cloning.design as design
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.io as cloning_io
import paulssonlab.cloning.primers as primers
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.thermodynamics as thermodynamics
import paulssonlab.cloning.viennarna as viennarna
import paulssonlab.cloning.workflow as workflow

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
geneious_sessionmaker = geneious.connect(**config["geneious"])

In [ ]:
reg = registry.Registry(
    gc,
    config["registry"]["folder"],
    geneious_sessionmaker=geneious_sessionmaker,
    geneious_folder="registry",
)

# Config

In [ ]:
olib_oligos = reg[("oLIB", "oligos")]
plib_plasmids = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]
flib_fragments = reg[("fLIB", "fragments")]
part_types = reg[("fLIB", "fragments", "Part types")]

# Scratch

In [ ]:
s = reg.eval_command("@Gib(pLIB125<oLIB376,oLIB377>, pLIB320<oLIB378,oLIB379>)")["_seq"]

In [ ]:
s

In [ ]:
prod = cloning_io.read_file("/Users/jacob/Downloads/pLIB320 PCR Product.gb")
uns_a = cloning_io.read_file("/Users/jacob/Downloads/Res_UNS_A.gb")
uns_e = cloning_io.read_file("/Users/jacob/Downloads/Res_UNS_E.gb")

In [ ]:
prod

In [ ]:
uns_a

In [ ]:
reg.get("ChlorR")["_seq"]

In [ ]:
uns_a

In [ ]:
workflow.re_digest_part(uns_a, Restriction.BsaI)

In [ ]:
workflow.re_digest_part(uns_e, Restriction.BsaI)

In [ ]:
tu = sequence.assemble(
    [
        workflow.re_digest_part(uns_a, Restriction.BsaI),
        reg.get("J23116_AB")["_seq"],
        reg.get("B0032m_BC")["_seq"],
        reg.get("ChlorR")["_seq"],
        reg.get("L3S1P13")["_seq"],
        workflow.re_digest_part(uns_e, Restriction.BsaI),
    ],
    method="goldengate",
)

In [ ]:
plasmid = sequence.assemble([tu, prod], method="gibson")

In [ ]:
reg.get("pLIB320")["_seq"].seq_lower()

In [ ]:
prod.seq_lower()

In [ ]:
plasmid.seq_lower()